Nos conectamos al drive, donde esta nuestro dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Tratamiento de las imagenes para las CNN**

In [2]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

Recuperamos la direccion del dataset y el haarcascade

In [3]:
dataset_path = "/content/drive/My Drive/valVGGface2"
face_cascade = cv2.CascadeClassifier('/content/drive/My Drive/haarcascade_frontalface_default.xml')

Parametros que utilizaremos para el tratamiento de datos

In [4]:
IMG_SIZE = (224, 224) #dimension de la redimension
#listas para almacenar imagenes(X), etiquetas(Y) y clases(classes)
X = []
Y = []
classes = []

Procesaremos las imagenes de cada persona y llenaremos X, Y y classes con datos correspondientes

In [5]:
#detectamos y procesamos cada imagen
for person_name in os.listdir(dataset_path): #recuperamos el nombre de la persona, vendra siendo el nombre de la carpeta
    person_path = os.path.join(dataset_path, person_name) #recuperamos la direccion de la carpeta de la persona
    if os.path.isdir(person_path): #verifica que existe la carpeta
        classes.append(person_name) #agregamos el nombre de las personas a las clases
        for img_name in os.listdir(person_path): #recuperamos cada imagen de la carpeta de la persona
            img_path = os.path.join(person_path, img_name) #obtenemos la direccion de la imagen de cada persona
            img = cv2.imread(img_path) #obtenemos la imagen
            if img is not None: #verifica que la imagen NO es None
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #pasamos la imagen a escala de grises
                faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5) #obtenemos la cara de la persona que esta en la imagen
                for (x, y, w, h) in faces:
                    cropped_face = gray[y:y+h, x:x+w] #usamos la imagen en escala de grises
                    resized_face = cv2.resize(cropped_face, IMG_SIZE) #redimensionamos a 224x224
                    X.append(resized_face / 255.0)  #normalizamos entre 0-1
                    Y.append(classes.index(person_name))

Pasamos X y Y a arrays de numpy

In [6]:
X = np.array(X)
Y = np.array(Y)

Redimensionamos las imagenes obtenidas(X) para que tengan un solo canal

In [7]:
#redimensionamos para que las imagenes tengan 1 canal
X = X.reshape(X.shape[0], IMG_SIZE[0], IMG_SIZE[1], 1)

Convertimos las etiquetas a formas categoricas

In [8]:
#convertimos las etiquetas a formato categorico (one-hot encoding)
Y = to_categorical(Y, num_classes=len(classes))

Ahora dividimos los datos que tenemos, en entrenamiento y validacion

In [10]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

In [11]:
#imprimimos las dimensiones de los conjuntos de datos
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {Y_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"y_val shape: {Y_val.shape}")

X_train shape: (3000, 224, 224, 1)
y_train shape: (3000, 12)
X_val shape: (751, 224, 224, 1)
y_val shape: (751, 12)


# **Creacion del modelo**

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

Parametros que utilizaremos durante la creacion del modelo

In [15]:
#parametros del modelo
input_shape = (224, 224, 1)  #imagenes en escala de grises (224x224x1)
num_classes = len(classes)  #numero de clases en classes

Creamos el modelo CNN

In [16]:
modelo = Sequential([
    #primera capa convolucional
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    BatchNormalization(),
    #primera capa maxpooling
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    #segunda capa convolucional
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    #segunda capa maxpooling
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    #tercera capa convolucional
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    #tercera capa maxpooling
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    #Aplanado (Flatten)
    Flatten(),

    #primera capa completamente conectada (256 unidades)
    Dense(256, activation='relu'),
    BatchNormalization(axis=1),
    Dropout(0.5),

    #segunda capa completamente conectada (200 unidades)
    Dense(200, activation='relu'),
    BatchNormalization(axis=1),
    Dropout(0.5),

    #tercera capa completamente conectada (200 unidades)
    Dense(150, activation='relu'),
    BatchNormalization(axis=1),
    Dropout(0.5),

    #cuarta capa completamente conectada (200 unidades)
    Dense(100, activation='relu'),
    BatchNormalization(axis=1),
    Dropout(0.5),

    #capa de salida (Softmax para clasificacion)
    Dense(num_classes, activation='softmax')
])

Compilamos el modelo

In [17]:
modelo.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # Entropía cruzada categórica
    metrics=['accuracy']
)

Veamos un resumen de nuestro modelo

In [18]:
modelo.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 222, 222, 32)        │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 222, 222, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 109, 109, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_9                │ (None, 52, 52, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 256)                 │      22,151,424 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_10               │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 200)                 │          51,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_11               │ (None, 200)                 │             800 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 200)                 │              

 Total params: 22,345,678 (85.24 MB)

 Trainable params: 22,343,818 (85.23 MB)

 Non-trainable params: 1,860 (7.27 KB)

Entrenamos nuestro modelo

In [19]:
history = modelo.fit(
    X_train, Y_train,  #datos de entrenamiento y etiquetas
    validation_data=(X_val, Y_val),  #datos de validacion y etiquetas
    epochs=50,  #numero de epocas
    batch_size=32,  #tamano del lote
    verbose=1
)

Epoch 1/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 56s 386ms/step - accuracy: 0.1275 - loss: 3.3378 - val_accuracy: 0.1052 - val_loss: 10.0444
Epoch 2/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 38s 61ms/step - accuracy: 0.2754 - loss: 2.4077 - val_accuracy: 0.1265 - val_loss: 4.9459
Epoch 3/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 6s 63ms/step - accuracy: 0.3419 - loss: 2.0480 - val_accuracy: 0.1917 - val_loss: 3.8570
Epoch 4/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 11s 66ms/step - accuracy: 0.3930 - loss: 1.8462 - val_accuracy: 0.1824 - val_loss: 4.4793
Epoch 5/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 6s 63ms/step - accuracy: 0.4647 - loss: 1.6422 - val_accuracy: 0.3862 - val_loss: 2.0727
Epoch 6/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 6s 62ms/step - accuracy: 0.5089 - loss: 1.5122 - val_accuracy: 0.5726 - val_loss: 1.3174
Epoch 7/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 6s 64ms/step - accuracy: 0.5643 - loss: 1.3485 - val_accuracy: 0.4780 - val_loss: 1.6185
Epoch 8/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 6s 63ms/step - accuracy: 0.5740 - loss: 1.2815 - val_accuracy: 0.635

Veamos el Loss y el Accuracy del modelo entrenado

In [20]:
loss, accuracy = modelo.evaluate(X_val, Y_val)
print(f"Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8964 - loss: 0.4552
Loss: 0.5390, Accuracy: 0.8842


Guardemos el modelo en un archivo .h5

In [22]:
modelo.save("reconocimientoFacialModelV2.h5")

In [23]:
classes

['n000148',
 'n000082',
 'guido',
 'n000106',
 'n000078',
 'n000040',
 'n000129',
 'n000029',
 'n000009',
 'n000001',
 'n000178',
 'n000149']